In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

from torch.utils.data import DataLoader

In [2]:
# 1 Data Transformer
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,),(0.5,))])

# 2 Create Train Dataset
trainset = torchvision.datasets.MNIST(root='./data', train =True,
                                    download = True, transform = transform)
trainloader = DataLoader(trainset, batch_size =64, shuffle =True)

#3 Create Test Dataset
testset = torchvision.datasets.MNIST(root = "./data", train = False,
                                     download = True, transform = transform)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super(NeuralNetwork,self).__init__()

        self.fc1 = nn.Linear(28*28,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(128,10)


    def forward(self,x):
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)

In [7]:
net = NeuralNetwork().to(device)
optimizer = optim.Adam(net.parameters(),lr = 0.001)
criterion = nn.CrossEntropyLoss().to(device)
epoch_size = 5

In [8]:
net.train()

for epoch in range(epoch_size):
    epoch_loss = 0
    correct_predictions = 0
    total_predictions = 0
    for data in trainloader:
        
        
        X, y = data[0].to(device), data[1].to(device)
    
        preds = net(X)
        loss = criterion(preds,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_loss+= loss

        #Calculate accuracy

        _, predicted = torch.max(preds,1)
        correct_predictions += (predicted ==y).sum().item()
        total_predictions+= y.size(0)

    accuracy = correct_predictions / total_predictions

    average_loss = epoch_loss / len(trainloader)

    print(f"Epoch {epoch+1} Loss: {average_loss:.4f} Accuracy: {accuracy:.4f}")
    


Epoch 1 Loss: 0.3274 Accuracy: 0.8960
Epoch 2 Loss: 0.1486 Accuracy: 0.9539
Epoch 3 Loss: 0.1115 Accuracy: 0.9648
Epoch 4 Loss: 0.0896 Accuracy: 0.9717
Epoch 5 Loss: 0.0757 Accuracy: 0.9766


In [21]:
# Convert the model to quantized version

quantized_model = torch.quantization.quantize_dynamic(
    net.to('cpu'), # Model
    {torch.nn.Linear}, # The layers to be quantized
    dtype=torch.qint8 # Quantization data type (int8)
)

In [22]:

def test_model(net, testloader, criterion, device):
    net.eval()  # modeli değerlendirme moduna al
    test_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # geri yayılım işlemi olmadan test et
        for data in testloader:
            X, y = data[0].to(device), data[1].to(device)
            preds = net(X)
            loss = criterion(preds, y)
            test_loss += loss.item()

            # Doğruluğu hesapla
            _, predicted = torch.max(preds, 1)
            correct_predictions += (predicted == y).sum().item()
            total_predictions += y.size(0)

    accuracy = correct_predictions / total_predictions
    average_loss = test_loss / len(testloader)
    print(f"Test Loss: {average_loss:.4f} Accuracy: {accuracy:.4f}")
    return average_loss, accuracy

In [23]:
net = net.to('cpu')
test_model(net,testloader,criterion,'cpu')

Test Loss: 0.1004 Accuracy: 0.9696


(0.10041138160511091, 0.9696)

In [24]:
quantized_model.eval()
test_model(quantized_model,testloader,criterion,"cpu")

Test Loss: 0.0999 Accuracy: 0.9695


(0.09989340358656967, 0.9695)